In [1]:
library(refund) # functional regression models for comparison
library(FuncNN) # neural networks with functional input
library(FDboost) # Boosting functional regression
library(tidyverse) # data wrangling
library(ggplot2) # plotting

# data import
dta <- readRDS("../data/data_comb.RDS")
names(dta)

Loading required package: mboost

Loading required package: parallel

Loading required package: stabs

This is FDboost 0.3-1. 

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::%+%()   masks mboost::%+%()
✖ tidyr::extract() masks mboost::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "id"              "cond"            "side"            "step"           
 [5] "age"             "ht"              "wt"              "sex"            
 [9] "ankle_accl_ap"   "ankle_accl_ml"   "ankle_accl_vt"   "ankle_angle_ap" 
[13] "ankle_angle_ml"  "ankle_angle_vt"  "ankle_moment_ap" "ankle_moment_ml"
[17] "ankle_moment_vt" "ankle_vel_ap"    "ankle_vel_ml"    "ankle_vel_vt"   
[21] "com_grf_ap"      "com_grf_ml"      "com_grf_vt"      "hip_accl_ap"    
[25] "hip_accl_ml"     "hip_accl_vt"     "hip_angle_ap"    "hip_angle_ml"   
[29] "hip_angle_vt"    "hip_moment_ap"   "hip_moment_ml"   "hip_moment_vt"  
[33] "hip_vel_ap"      "hip_vel_ml"      "hip_vel_vt"      "knee_accl_ap"   
[37] "knee_accl_ml"    "knee_accl_vt"    "knee_angle_ap"   "knee_angle_ml"  
[41] "knee_angle_vt"   "knee_moment_ap"  "knee_moment_ml"  "knee_moment_vt" 
[45] "knee_vel_ap"     "knee_vel_ml"     "knee_vel_vt"     "cycle"          
[49] "study"           "split_set"

In [2]:
# Get variables---------------------------------------------------------------

response_vars <- names(dta)[grep("grf|knee_moment|hip_moment|ankle_moment", 
                                 names(dta))]
pred_vars <- names(dta)[grep("angle|vel|accl", names(dta))]

# Restructure data -----------------------------------------------------------

dta_mat <- dta[map_lgl (dta, is.matrix)]
dta_x <- dta_mat[grepl ("accl|vel|angle", names (dta_mat))]
dta_y <- dta_mat[grepl ("grf|knee_moment|hip_moment|ankle_moment", 
                        names(dta_mat))]
axes <- c("ap", "ml", "vt")
prednames <- unique (str_remove_all(pred_vars, "_ap|_ml|_vt"))
outnames <- unique (str_remove_all(response_vars, "_ap|_ml|_vt"))

set.seed(42)

train_ind <- sample(1:nrow(dta_y[[1]]), floor(0.8*nrow(dta_y[[1]])))
test_ind <- setdiff(1:nrow(dta_y[[1]]), train_ind)

# define final data for comparison
x_sca <- model.matrix(~ -1 + ., data = as.data.frame(
  dta[c("study", "sex", "cond", "side", "step", "age", "ht", "wt")]
))

y_train <- dta_y$ankle_moment_ap[train_ind,]
x_train_fun <- lapply(dta_x, function(x) x[train_ind,])
x_train_sca <- x_sca[train_ind,]

y_test <- dta_y$ankle_moment_ap[test_ind,]
x_test_fun <- lapply(dta_x, function(x) x[test_ind,])
x_test_sca <- x_sca[test_ind,]


In [3]:
# Another split for validation
train_train_ind <- sample(1:nrow(x_train_fun[[1]]), 
                          floor(0.8*nrow(x_train_fun[[1]])))
train_val_ind <- setdiff(1:nrow(x_train_fun[[1]]), train_train_ind)

# for pffr
cycle <- dta$cycle
train <- dta[names(dta)!="cycle"]
train <- lapply(train, function(x) if(is.null(dim(x))) x[train_ind] else 
  x[train_ind,])
test <- dta[names(dta)!="cycle"]
test <- lapply(test, function(x) if(is.null(dim(x))) x[test_ind] else 
  x[test_ind,])


In [4]:

################ PFFR ####################
print("Starting pffr model...") 
response <- response_vars[1]

# Create formula
# Create formula with ffpc for low-rank functional predictors
form <- paste(response, " ~ 1 + ", paste(
  paste0("ffpc(", pred_vars,
         ", yind=cycle, xind=cycle, npc=", c(2, 2, 4), ")"),  # Adjust npc for each predictor
  collapse = " + "),
  " + age",
  " + ht",
  " + wt",
  " + sex",
  "+ s(cond, bs = 're')",
  "+ s(id, bs = 're')"
)


train <- as.data.frame(lapply(train, function(x) if(!is.null(dim(x))) 
  return(I(x)) else x))
train$id <- as.factor(train$id)
train$cond <- as.factor(train$cond)
train$sex <- as.factor(train$sex)
test <- as.data.frame(lapply(test, function(x) if(!is.null(dim(x))) 
  return(I(x)) else x))
test$id <- as.factor(test$id)
test$cond <- as.factor(test$cond)
test$sex <- as.factor(test$sex)

                            
# initialize the model
m <- pffr(as.formula(form),
          yind = 1:101,
          algorithm = "bam",
          data = train)

prediction_pffr <- m %>% predict(test)


[1] "Starting pffr model..."


In [5]:

saveRDS(prediction_pffr, file="../results/prediction_pffr.RDS")

rm(m, prediction_pffr); gc()


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2973756,158.9,4765674,254.6,4765674,254.6
Vcells,11647726,88.9,84892471,647.7,106115588,809.6


In [6]:
# Create formula
form <- paste(response, " ~ 1")

# initialize the model
mint <- pffr(as.formula(form),
             yind = 1:101,
             algorithm = "bam",
             data = train)

prediction_intercept <- mint %>% predict(test[c("id")])

saveRDS(prediction_intercept, file="../results/prediction_intercept.RDS")

rm(mint, prediction_intercept); gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2980370,159.2,4765674,254.6,4765674,254.6
Vcells,11662713,89.0,67913977,518.2,106115588,809.6


In [7]:
# Get variables---------------------------------------------------------------

response_vars <- names(dta)[grep("grf|knee_moment|hip_moment|ankle_moment", 
                                 names(dta))]
pred_vars <- names(dta)[grep("angle|vel|accl", names(dta))]

# Restructure data -----------------------------------------------------------

dta_mat <- dta[map_lgl(dta, is.matrix)]
dta_x <- dta_mat[grepl("accl|vel|angle", names(dta_mat))]
dta_y <- dta_mat[grepl("grf|knee_moment|hip_moment|ankle_moment", 
                       names(dta_mat))]

set.seed(42)
train_ind <- sample(1:nrow(dta_y[[1]]), floor(0.8 * nrow(dta_y[[1]])))
test_ind <- setdiff(1:nrow(dta_y[[1]]), train_ind)

# Define data for PFFR
x_sca <- model.matrix(~ -1 + ., data = as.data.frame(
  dta[c("study", "sex", "cond", "side", "step", "age", "ht", "wt")]
))
y_train <- dta_y$ankle_moment_ap[train_ind,]
x_train_fun <- lapply(dta_x, function(x) x[train_ind,])
x_train_sca <- x_sca[train_ind,]

y_test <- dta_y$ankle_moment_ap[test_ind,]
x_test_fun <- lapply(dta_x, function(x) x[test_ind,])
x_test_sca <- x_sca[test_ind,]

# Prepare training and testing data for PFFR
cycle <- dta$cycle
train <- dta[names(dta) != "cycle"]
train <- lapply(train, function(x) if (is.null(dim(x))) x[train_ind] else x[train_ind,])
test <- dta[names(dta) != "cycle"]
test <- lapply(test, function(x) if (is.null(dim(x))) x[test_ind] else x[test_ind,])

# PFFR Model ----------------------------------------------------------------
print("Starting PFFR model...") 
response <- response_vars[1]

# Create formula with ffpc for low-rank functional predictors
form <- paste(response, " ~ 1 + ", paste(
  paste0("ffpc(", pred_vars, ", yind=cycle, xind=cycle, npc=", c(2, 2, 4), ")"),  
  collapse = " + "),
  " + age",
  " + ht",
  " + wt",
  " + sex",
  "+ s(cond, bs = 're')",
  "+ s(id, bs = 're')"
)

train <- as.data.frame(lapply(train, function(x) if (!is.null(dim(x))) return(I(x)) else x))
train$id <- as.factor(train$id)
train$cond <- as.factor(train$cond)
train$sex <- as.factor(train$sex)
test <- as.data.frame(lapply(test, function(x) if (!is.null(dim(x))) return(I(x)) else x))
test$id <- as.factor(test$id)
test$cond <- as.factor(test$cond)
test$sex <- as.factor(test$sex)

# Initialize the PFFR model
m <- pffr(as.formula(form), yind = 1:101, algorithm = "bam", data = train)

# Predictions
prediction_pffr <- predict(m, newdata = test)
saveRDS(prediction_pffr, file = "../results/prediction_pffr.RDS")

rm(m, prediction_pffr); gc()

# Comparison and Visualization ------------------------------------------------

# Load predictions and true values
results <- c(list(y_test), lapply(list.files("results", full.names = TRUE), 
                                  function(x) as.matrix(readRDS(x))))
nams <- c("Truth", "PFFR")
resultsDF <- do.call("rbind", lapply(1:length(results), function(i) 
  data.frame(value = c(results[[i]]), time = rep(1:101, each = nrow(y_test)),
             obs = rep(1:nrow(y_test), 101), what = nams[i])))

# Calculate RMSE for PFFR model
rmse_pffr <- mean(sqrt(rowSums((results[[2]] - results[[1]])^2)))
rmseDF <- data.frame(rmse = paste0("RMSE: ", round(rmse_pffr, 4)), what = "PFFR", obs = 1)

# Plot
palette <- RColorBrewer::brewer.pal(8, "Dark2")
palette <- rgb(col2rgb(palette)[1, ] / 255, col2rgb(palette)[2, ] / 255, col2rgb(palette)[3, ] / 255, alpha = 0.25)

par(mfrow = c(1, 2), cex = 0.8)
for (method in unique(resultsDF$what)) {
  single_result <- subset(resultsDF, what == method)
  matplot(t(matrix(single_result$value, ncol = 101)), 
          x = seq(0, 1, length.out = 101), 
          col = palette[as.integer(single_result$what)], 
          type = 'l', ylim = c(-1.1, 1.1), 
          xlab = 'Relative Time', ylab = 'Value', main = method, bty = "n")
  
  if (method == "PFFR") {
    text(x = 0.05, y = -0.9, labels = rmseDF$rmse, pos = 4)
  }
}


[1] "Starting PFFR model..."


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2981975,159.3,4765674,254.6,4765674,254.6
Vcells,11674372,89.1,82266808,627.7,106115588,809.6


ERROR: Error in results[[2]]: subscript out of bounds
